# Farmer Ted -- A Stochastic Programming Example

Farmer Ted can plant corn, wheat, and beans on his 500 acres. He requires 200 tons of wheat and 240 tons of corn for his cattle. Wheat and corn can be grown on his own land or bought from a wholesaler. Any excess production can be sold for \\$170/ton for wheat and \\$150/ton for corn. Any shortfall must be bought from the wholesaler at a cost of \\$238/ton for wheat and \\$210/ton for corn. Farmer Ted can also grow beans. Beans sell at \\$36/ton for the first 6000 tons. Due to economic quotas on bean production, beans in excess of 6000 tons can only be sold at \\$10/ton.

The following table contains the relevant data for the problem:

|                     | Wheat | Corn | Beans          |
|---------------------|-------|------|----------------|
| Yield (T/acre)      | 2.5   | 3    | 20             |
| Plant cost ($/acre) | 150   | 230  | 260            |
| Sell price          | 170   | 150  | 36 (<= 6000T)  |
|                     |       |      | 10 (> 6000T)   |
| Purchase price      | 238   | 210  | N/A            |
| Minimum required    | 200   | 240  | N/A            |

## Linear programming model

In [1]:
prod = ["W","C","B"]
yield = Dict(zip(prod,[2.5 3 20]))
cost = Dict(zip(prod,[150 230 260]))
sell_price = Dict(zip(prod,[170 150 36]))
bean_price_low = 10

using JuMP, Clp

m = Model(Clp.Optimizer)
set_optimizer_attribute(m,"LogLevel",0)

@variable(m, x[prod] >= 0)
@variable(m, w[prod] >= 0)
@variable(m, y[prod[1:2]] >= 0)
@variable(m, e >= 0)

@objective(m, Max, -sum(cost[i]*x[i] for i in prod) - 
    238y["W"] - 210y["C"] + 
    sum(sell_price[i]*w[i] for i in prod) +
    bean_price_low*e)


@constraint(m, sum(x) <= 500)
@constraint(m, 2.5x["W"]  + y["W"] - w["W"] == 200)
@constraint(m, 3x["C"]  + y["C"] - w["C"] == 240)
@constraint(m, 20x["B"] - w["B"] - e == 0)
@constraint(m, w["B"] <= 6000)

optimize!(m)

println("Plant: ", value.(x))
println("Buy: ", value.(y))
println("Sell (normal price): ", value.(w))
println("Sell beans (lower price): ", value.(e))
println("Profit: ", objective_value(m))


Plant: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["W", "C", "B"]
And data, a 3-element Vector{Float64}:
 120.0
  80.0
 300.0
Buy: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["W", "C"]
And data, a 2-element Vector{Float64}:
 0.0
 0.0
Sell (normal price): 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["W", "C", "B"]
And data, a 3-element Vector{Float64}:
  100.0
    0.0
 6000.0
Sell beans (lower price): 0.0
Profit: 118600.0


## Stochastic Programming model

In [2]:
scen = [1,2,3]
yield_frac = Dict(zip(scen, [1.2 1 0.8]))

m = Model(Clp.Optimizer)
set_optimizer_attribute(m,"LogLevel",0)

@variable(m, x[prod] >= 0)
@variable(m, w[prod,scen] >= 0)
@variable(m, y[prod[1:2],scen] >= 0)
@variable(m, e[scen] >= 0)

@objective(m, Max, -sum(cost[i]*x[i] for i in prod) +
    (1/3)*sum(-238y["W",s] - 210y["C",s] + 
    sum(sell_price[i]*w[i,s] for i in prod) +
    bean_price_low*e[s] for s in scen))

@constraint(m, sum(x) <= 500)
@constraint(m, wheat[s in scen], yield_frac[s]*2.5*x["W"]  + y["W",s] - w["W",s] == 200)
@constraint(m, corn[s in scen], yield_frac[s]*3*x["C"]  + y["C",s] - w["C",s] == 240)
@constraint(m, bean[s in scen], yield_frac[s]*20*x["B"] - w["B",s] - e[s] == 0)
@constraint(m, bean_bound[s in scen], w["B",s] <= 6000)

optimize!(m)

println("Plant: ", value.(x))
println("Buy: ", value.(y))
println("Sell (normal price): ", value.(w))
println("Sell beans (lower price): ", value.(e))
println("Profit: ", objective_value(m))



Plant: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["W", "C", "B"]
And data, a 3-element Vector{Float64}:
 170.0
  80.00000000000001
 250.0
Buy: 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["W", "C"]
    Dimension 2, [1, 2, 3]
And data, a 2×3 Matrix{Float64}:
 0.0  0.0   0.0
 0.0  0.0  47.99999999999995
Sell (normal price): 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["W", "C", "B"]
    Dimension 2, [1, 2, 3]
And data, a 3×3 Matrix{Float64}:
  310.0                225.00000000000006   140.00000000000006
   47.99999999999996     0.0                  0.0
 6000.0               5000.0               4000.0
Sell beans (lower price): 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{Float64}:
 0.0
 0.0
 0.0
Profit: 108390.0
